In [1]:
import requests
import pandas as pd
import urllib.parse
import folium

In [2]:
url = "https://civilization.fandom.com/wiki/List_of_wonders_in_Civ5"
r = requests.get(url)
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list

In [3]:
result = pd.DataFrame([], columns = ['Wonder'])

In [4]:
for n in range(len(df_list)-3):
    if 'Wonder' in df_list[n].columns:
        result = pd.concat([result, df_list[n]], ignore_index=True).drop(columns=['Requirements', 'Production', 'Effects'])
result['Wonder'] = result['Wonder'].str.replace('[^a-zA-Z ]', '')

C:\Users\mall2\AppData\Local\Temp\ipykernel_11844\3390244541.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  result['Wonder'] = result['Wonder'].str.replace('[^a-zA-Z ]', '')


In [5]:
places = "http://www.kalle-online.net/blog/wonders-world-civilization-5/"
p = requests.get(places)
df_list_pls = pd.read_html(p.text) # this parses all the tables in webpages to a list

In [6]:
def categorise(row):  
    if 'Destroyed' in row['Real Wonders of the World']:
        return 'Yes'
    return 'No'
pls = df_list_pls[1]
pls['Destroyed'] = pls.apply(lambda row: categorise(row), axis=1)
pls[['Wonder', 'City', 'Country']] = pls['Civilization 5 Wonders'].str.split(',', n=2, expand=True)
pls = pls.drop(columns = ['Real Wonders of the World', 'Civilization 5 Wonders'])
pls.loc[pls['Country'].isna(), 'Country'] = pls.City
pls.loc[pls['Country'] == pls['City'], 'City'] = 'Not defined'
#pls = pls[['Wonder', 'City', 'Country', 'Destroyed']]

In [7]:
# Comparing 2 pages on the content
df_diff = pd.concat([result,pls]).drop_duplicates(subset=['Wonder'], keep=False)
df_diff

,Wonder,Destroyed,City,Country
20,Hermitage,No,Saint Petersburg,Russia Only being a national wonder in Civili...
23,International Space Station,No,Orbit,Earth
32,Oxford University,No,Oxford,England Only being a national wonder in Civil...


In [8]:
pls['Country'] = pls['Country'].str.split(' ', n=2, expand=True)[1]

In [9]:
pls.drop(pls[pls['Country'].isna()].index, inplace = True)
pls.drop(pls[pls['Country'] == 'Earth'].index, inplace = True)
pls.loc[46, 'Wonder'] = 'Temple of Artemis'
pls.loc[46, 'City'] = 'Selcuk'
pls.loc[46, 'Country'] = 'Turkey'
pls = pls.reset_index(drop=True)

In [10]:
def categorise(Address):  
    if 'Not defined' in Address['City']:
        return (Address['Wonder'] + ', ' + Address['Country'])
    return (Address['Wonder'] + ', ' + Address['City'] + ', ' + Address['Country'])
pls['Address'] = pls.apply(lambda Address: categorise(Address), axis=1)

In [11]:
def location(Address):  
    address = Address['Address']
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    if not response:
        Address['latitude'] = 'Not defined'
        Address['longitude'] = 'Not defined'
    else :
        Address['latitude'] = float(response[0]["lat"])
        Address['longitude'] = float(response[0]["lon"])
    return (Address)
to_map = pls.apply(location, axis=1)
pd.set_option("display.precision", 3)

In [13]:
to_map.iloc[13, 5:7] = [31.208, 29.909]
to_map.iloc[14, 5:7] = [31.125, 29.531]
to_map.iloc[25, 5:7] = [37.022, 27.253]
to_map.iloc[28, 5:7] = [38.286, 22.301]
to_map.iloc[33, 5:7] = [32.075, 118.725]
to_map.iloc[36, 5:7] = [28.656, 77.243]
to_map['Visited'] = 'No'
to_map = to_map[['Wonder', 'City', 'Country', 'Address', 'latitude', 'longitude', 'Visited', 'Destroyed']]

In [15]:
lat = to_map['latitude']
lon = to_map['longitude']
name = to_map['Address']
dest = to_map['Destroyed']
visit = to_map['Visited']
map = folium.Map(location=[30,5], zoom_start = 2, title = 'Stamen')
def color_change(dest, visit):
    if((dest == 'Yes') & (visit == 'No')):
        return('gray')
    elif((visit == 'No') & (dest == 'No')):
        return('red')
    else:
        return('green')
for lat, lon, name, dest, visit in zip(lat, lon, name, dest, visit):
    folium.Marker(location=[lat, lon], popup=str(name), icon=folium.Icon(color = color_change(dest, visit))).add_to(map) 
map

In [17]:
to_map.to_csv("C:\\Users\\mall2\\Downloads\\Wonders.csv")
map.save("map1.html")